## Менеджер моделей в Django
Интерфейс взаимодействия с базой данных.

> По умолчанию, каждая модель получает менеджер `objects`.


## Создание менеджеров моделей
- Менеджер, объявленный первым в модели - Дефолтный
- Для указания другой модели дефолтной используется - [[Python/Frameworks/Django/02 Models/class Meta#^e1895c|default_manager_name]]
### `Model.MY_MANAGER.all()`

```python
from django.db import models
class PublishedManager(models.Manager):
    def get_queryset(self):
        return super().get_queryset().filter(published=True)
        
class Post(models.Model):
    title = models.CharField(max_length=200)
    content = models.TextField()
    published = models.BooleanField(default=False)
    
    objects = models.Manager()  # The default manager.
    published_objects = PublishedManager()  # Our custom manager.
    
# Get all published posts.
published_posts = Post.published_objects.all()
```

### `Model.objects.MY_MANAGER()`

```python
from django.db import models

class PostManager(models.Manager):
    def my_manager(self):
        return self.filter(title__startswith='A')

class Post(models.Model):
    title = models.CharField(max_length=200)
    content = models.TextField()
    objects = PostManager()  # Our custom manager.
```

```python
# Get all posts that title starts with 'A'
a_posts = Post.objects.my_manager()
```

## Создание объектов

```python
# Сохранение при вызове метода save()
news = News(
  title='новость1',
  content='Данные'
)
news.save()
```
```python
# Сохранение при вызове метода save()
n = News()
n.title='title'
n.content='content'
n.save()
```
```python
# Автоматическое сохранение
n = News.objects.create(
	title='title',
	content='content'
)  # автоматически вызывается save()
```


## Чтение объектов

Наборы запросов QuerySet вычисляются вычисляются:
- при первом их прокручивании в цикле;
- при их нарезке, например Post.objects.all()[:3];
- при их консервации в поток байтов или кешировании;
- при вызове на них функций repr() или len();
- при вызове на них функции list() в явной форме;
- при их проверке в операциях bool(), or, and или if

### Manager (objects)

## Свойство count
Количество записей

```python
Post.objects.filter(pub_date__month__gt=6, pub_date__year=1854).count()
```


### Методы

#### all()

`all()` возвращает QuerySet, содержащий все объекты в базе данных для данной модели.
```python
# Все строки
all_posts = Post.objects.all()
# Срез записей
all()[:3]
```

#### aggregate()

Применяет аггрегирующую функцию к полю или всей таблице
> Возвращает только значение, в отличие от [[annotate()|annotate()]]
> Именованные параметры позволяют выполнять вычисления над аггрегатными функциями
```python
from django.db.models import Avg, Count, Max, Min, Sum
Post.objects.aggregate(
	Count('pk'),
	min=Min('id'),
	max=Max("id"),
) # {'min':1, 'max':9, 'pk__count':9}
# Вычисления
News.objects.aggregate(
	my_sum=Sum('total')-Count('price')
)
# {'my_sum': 999}
```

#### annotate()
Добавляет для каждого экземпляра класса новое свойство и возвращает QuerySet\
в отличие от `aggregate`, который возвращает значение

> `annotate(cnt=Count('news'), filter=F('news__is_published'))`

```python
from django.db.models import Avg, Count, Max, Min, Sum
Avg('field')
Count('field')
Max('field')
Min('field')
Sum('field')
```
```python
newUsers = User.objects.annotate(Sum('posts'))
newUser.posts__sum  # 999
newUsers = User.objects.annotate(count_posts = Sum('posts'))
newUser.count_posts  # 999
News.objects.annotate(cnt=Count('pk')).filter(cnt__gt=0)
```

#### Вычисляемые поля
```python
from django.db.models import F
for item in Goods.objects.annotate(half_price=F('price')/2):
	print(item.title, item.half_price)
```

#### count()
Возвращает количество объектов в QuerySet.

```python
count = Post.objects.count()
```

#### dates()
Возвращает QuerySet, содержащий даты из указанного поля.

```python
# Получение уникальных дат из поля
dates(
	<имя поля>,
	<часть даты>    # year, month, day
	[, order='ASC']
)
```

#### datetimes()
Возвращает QuerySet, содержащий объекты datetime из указанного поля.

```python
post_times = Post.objects.datetimes('publish_date', 'hour')
```

```python
# Получение уникальных дат/времени из поля
datetimes(
	<имя поля>,
	<часть даты>    # year, month, day, hour, minute, second
	[, order='ASC']
	[, tzinfo=None]
)
```

#### distinct()
Возвращает QuerySet, содержащий уникальные объекты.

```python
unique_authors = Post.objects.order_by('author').distinct('author')
```

#### earliest()
Возвращает самый ранний объект в QuerySet согласно указанному полю.

```python
earliest_post = Post.objects.earliest('publish_date')
```

#### exclude()
Возвращает QuerySet, который исключает объекты, соответствующие переданному набору критериев поиска.

```python
non_published_posts = Post.objects.exclude(published=True)
```

#### exists()
Возвращает `True`, если QuerySet содержит хотя бы один объект, и `False` в противном случае.

```python
has_posts = Post.objects.filter(title='My Post').exists()
```

#### filter()
Возвращает QuerySet, содержащий объекты, соответствующие переданному набору критериев поиска.

В одном запросе можно объединить несколько условий
> Default: `AND`. Для `OR` - `Q()|Q()`

```python
filter().filter()
filter(condition, condition)
```

❗ Запросы с операциями поиска в полях формируются с использованием двух знаков подчеркивания, например `publish__year`, но те же обозначения также используются для обращения к полям ассоциированных моделей, например `author__username`.

```python
recent_posts = Post.objects.filter(publish_date__year=2023)  # Фильтр
```

#### Именованные условия
```python
field__contains='abc'      # содержит
field__icontains='abc'     # содержит / NonCaceSens
field__in=['a', 'b', 'c']  # IN
field__exact='abc'         # точное совпадение
field__iexact='abc'        # точное совпадение / NonCS
field__startswith='abc'    # Начинается с
field__istartswith='abc'   # Начинается с / NonCS
field__endswith='abc'      # Заканчивается на
field__iendswith='abc'     # Заканчивается на / NonCS
field__range=(start, end)  # BETWEEN
field__isnull=True         # IS NULL
field__gt=5      # Больше
field__gte=5     # Больше или равно
field__lt=5      # Меньше
field__lte=5     # Меньше или равно
field__regex=''  # Регулярка
field__iregex='' # Регулярка NonCS
field__date
field__year
field__month
field__day
field__week
field__week_day
field__quarter
field__hour
field__minute
field__second
```



#### first()
Возвращает первый объект в QuerySet или `None`, если QuerySet пуст.

```python
first_post = Post.objects.first()
```

#### get()
❗️ Всегда ожидает одну запись
> - Если нет результата - вызовет исключение `DoesNotExist`\
> - Если более одного результата - `MultipleObjectsReturned`.

```python
post = Post.objects.get(id=1)     # Одна строка
```

#### in_bulk()
Возвращает словарь, преобразуя каждый объект QuerySet в словарь с ключами из первичного ключа модели.

```python
posts = Post.objects.in_bulk([1, 2, 3])
```

```python
in_bulk(  # field_name в последовательности значений
	<последовательность значений>
	[, field_name='pk']
)
````

#### last()
Возвращает последний объект в QuerySet или `None`, если QuerySet пуст.

```python
last_post = Post.objects.last()
```

#### latest()
Возвращает самый поздний объект в QuerySet согласно указанному полю.

```python
latest_post = Post.objects.latest('publish_date')
```

#### order_by()
Ввозвращает QuerySet, отсортированный по указанным полям.

```python
posts = Post.objects.order_by('-publish_date')
```

#### prefetch_related()
Аналогичен `select_related()`, но он работает с "многие-ко-многим" и "многие-к-одному" связями.

> Ленивая загрузка

```python
posts = Post.objects.prefetch_related('tags')
```

Подобно `select_related()`, этот метод уменьшит количество запросов к базе данных при обращении к связанным объектам, таким как `post.tags.all()`.

```python
related = Post.objects.prefetch_related('author').all()
for post in related:
    tmp = f"{post.text} Автор {post.author.username}"
```

#### values_list()
Возвращает QuerySet, который вместо полных модельных объектов возвращает кортежи со значениями полей.

```python
post_values = Post.objects.values_list('id', 'title')
```

#### values()
Возвращает QuerySet, который вместо полных модельных объектов возвращает словари со значениями полей.

```python
post_values = Post.objects.values()
```

### Аггрегирующие функции

#### Avg('field')

```python
Avg( # Среднее значение
	<имя поля>
	[, output_field=FloatField()] # Тип результирующего поля
	[, filter=None]               # Фильтр Q
)
```

#### Count('field')

```python
Count( # Количество записей
	<имя поля>
	[, distinct=False]  # True - Только уникальные
	[, filter=None]     # Фильтр Q
)
```

#### Min('field')
```python
Min( # Минимальное значение
	<имя поля>
	[, output_field=None] # Тип результирующего поля
	[, filter=None]       # Фильтр Q
)
```

#### Max('field')

```python
Max( # Максимальное значение
	<имя поля>
	[, output_field=None] # Тип результирующего поля
	[, filter=None]       # Фильтр Q
)
```

#### Sum('field')

```python
Sum( # Сумма значений
	<имя поля>
	[, output_field=None] # Тип результирующего поля
	[, filter=None]       # Фильтр Q
)
```

#### StdDev('field')

```python
StdDev( # Стандартное отклонение
	<имя поля>
	[, samlple=None] # Стандартное отклонение выборки
	[, filter=None]  # Фильтр Q
)
```

#### Variance('field')

```python
Variance( # Дисперсия
	<имя поля>
	[, samlple=None] # Стандартное отклонение выборки
	[, filter=None]  # Фильтр Q
)
```

## Обновление объектов
`.update()` используется для обновления значений одного или нескольких полей для всех записей в QuerySet.
> - Возвращает количество обновленных записей.
> - Обновляет поля напрямую в базе данных

```python
Post.objects.filter(publish_year=2022).update(title='Updated title')
```

```python
post = Post.objects.get(id=1)
post.title = 'Updated title'
post.save()
```

## Удаление объектов

`.delete()` может быть применен к отдельному экземпляру модели или к QuerySet (набору объектов).
> Метод `.delete()` возвращает кортеж: `(int, dict[Type, int])`

#### Удаление отдельного объекта:
```python
post = Post.objects.get(id=1)
post.delete()
```

#### Удаление QuerySet:
```python
Post.objects.filter(publish_year=2022).delete()
```

## Прочие функции/Классы

### get_or_create()
Поиск записи и ее создание, в случае отсутствия

```python
News.objects.get_or_create(
	<набор фильтров
	[, defaults=None]
)
News.objects.get_or_create(name='Новость 1')
```

### update_or_create()
Поиск записи и ее создание, в случае отсутствия\
Если найдена, то добавляет в нее поля из defaults

```python
News.objects.update_or_create(
	<набор фильтров
	[, defaults=None]
)
News.objects.update_or_create(name='Новость 1')
```

### F()
Сравнение со значениями других полей

```python
from django.db.models import F
f = F('title')

for iNews.objects.filter(content__icontains=f):
	print(n.title, end=' ')
```
```python
news.views = F('views') + 1
```

### Q()
Позволяет создавать сложные запросы\
`&` И\
`|` ИЛИ\
`~` НЕ\

```python
from django.db.models import Q

q = Q(category__name='Категория1') | Q(category__name='Категория2')

for iNews.objects.filter(q):
	print(n.title, end=' ')
```

## Вложенные запросы

```python
from django.db.models import Subquery, OuterRef
Subquery(
	<вложенный набор записей
	[, output_field=None]
)
# Для ссылки на поля внешнего набора записей
OuterRef(<поле внешнего набора записей)
```
```python
from django.db.models import Exists, OuterRef
Exists(<вложенный набор записей)
```
```python
subquery = Exists(
	Bb.objects.filter(
		rubric=OuterRef('pk'),
		price__gt=l00000
	)
)
for r iRubric.objects
	.annotate(is_expensive=subquery)
	.filter(is_expensive=True):
		print(r.name)
```

## Объединения (union)

```python
union(
	<набор записей1,
	<набор записей2, ...
	[, all=False]  # True ## UNIOALL, False ## UNION
)
intersection(  # Записи, имеющиеся в каждом из наборов
	<набор записей1,
	<набор записей2, ...
)
difference(    # Записи в одном из наборов, но не в двух сразу
	<набор записей1,
	<набор записей2, ...
)
```

## Условные выражения СУБД

```python
Cast(
	<условие1, <условие2, ...,
	[, default=None]       # Значение по умолчанию
	[, output_field=None]  # Тип поля
)
```
```python
from django.db.models import Case, When, Value, Count, CharField
for cat iCategory.objects.annotate(
	cnt=Count('news'),
	cnt_s=Case(
		When(cnt__gte=5, then=Value('Много')),
		When(cnt__gte=5, then=Value('Много')),
		When(cnt__gte=5, then=Value('Много')),
		default=Value('Вообще нет'),
		output_field=CharField()
	)
):
	print(r.title, r.cnt_s)
```

## Функции СУБД
Классы из django.db.functions

### Coalesce
Возвращает первое значение, отличное от null\
Значения представляются строками, экземплярами или Value\
Все значения должны иметь одинаковый тип

```python
Coalesce('title', 'name', 'content')
```

### Greatest
Возвращает наибольшее значение из переданных\
Значения представляются строками, экземплярами или Value\
Все значения должны иметь одинаковый тип

```python
Greatest('title', 'name', 'content')
```

### Least
Возвращает наименьшее значение из переданных\
Значения представляются строками, экземплярами или Value\
Все значения должны иметь одинаковый тип

```python
Least('title', 'name', 'content')
```

### Cast
Принудительно приводит значение к типу\
Значение представляется строкой или экземпляром F\
Тип ## экземпляр Класса, представляющий поле соответствующего типа

```python
Cast(<значение, <тип)
```

### Concat
Возвращает результат конкатенации переданных строк\
Значения представляются строками, экземплярами или Value\
Все значения должны иметь строковый тип

```python
Concat('title', 'name', 'content')
```

### Lower
Преобразует значение к нижнему регистру\
Значение представляется строкой или экземпляром F

```python
Lower('name')
```

### Upper
Преобразует значение к верхнему регистру\
Значение представляется строкой или экземпляром F

```python
Upper('name')
```

### Length
Возращает длину строки\
Значение представляется строкой или экземпляром F

```python
Length('name')
```

### StrIndex
Номер вхождения подстроки в строку, либо 0\
Нумерация начинается с 1\
Значения представляются строками, экземплярами или Value

```python
StrIndex(<строка, <подстрока)
```

### Substr
Извлекает подстроку из строки\
Нумерация начинается с 1\
Значение представляется строкой или экземпляром F

```python
Substr('name', <позиция[, <длина])
```

### Left
Возвращает подстроку от начала значения

```python
Left(<значение, <длина)
```

### Right
Возвращает подстроку от конца значения

```python
Right(<значение, <длина)
```

### Replace
Замена подстроки в строке\
Если заменяющая строка не указана, то ''

```python
Replace(
	<значение,
	<заменяемая подстрока
	[, <заменяющая подстрока]
)
```

### Repeat
Повторяет значение раз

```python
Repeat(<значение, <кол-во повторов)
```

### LPad
Дополняет значение слева символом-заполнителем\
Если заполнитель не указан, используется пробел

```python
LPad(<значение, <длина[, <символ-заполнитель])
```

### RPad
Дополняет значение справа символом-заполнителем\
Если заполнитель не указан, используется пробел

```python
RPad(<значение, <длина[, <символ-заполнитель])
```

### Trim
Удаляет пробелы слева и справа

```python
Trim(<значение)
```

### LTrim
Удаляет пробелы слева

```python
Trim(<значение)
```

### RTrim
Удаляет пробелы справа

```python
Trim(<значение)
```

### Chr
Символ по коду

```python
Chr(<код символа)
```

### Ord
Код символа

```python
Ord(<символ)
```

### Now()
Текущие дата и время

```python
Now()
```

### Extract
Извлекает часть даты и возвращает число

```python
Extract(
	<значение даты или времени,
	<извлекаемая часть,
	[, tzinfo=None]      # Временная зона
)
# Извлекаемые значения:
# year, quarter, month, day, week, week_day, hour, minute, second
```

Упрощенные классы
```python
    ExtractYear(value[, tzinfo=None])
    ExtractQuater(value[, tzinfo=None])
    ExtractMonth(value[, tzinfo=None])
    ExtractDay(value[, tzinfo=None])
    ExtractWeek(value[, tzinfo=None])
    ExtractWeekDay(value[, tzinfo=None])
    ExtractHour(value[, tzinfo=None])
    ExtractMinute(value[, tzinfo=None])
    ExtractSecond(value[, tzinfo=None])
```

### Truпc
Значение даты времени до указанной конечной части\
Значение представляется экземпляром и должно принадлежать типу даты, времени или даты и времени

```python
Truпc(
	<значение даты или времени,
	<конечная часть,
	[, output_field=None] # Тип возвращаемого значения
	[, tzinfo=None]       # Временная зона
)
# Конечная часть:
# year, quarter, month, week, day, hour, minute, second
Trunc('published', 'second') # 10.11.2020 08:42:31
Trunc('published', 'year')   # 10.11.2020 00:00:00
```

Упрощенные классы
```python
    TruncYear(value[, tzinfo=None])
    TruncQuater(value[, tzinfo=None])
    TruncMonth(value[, tzinfo=None])
    TruncWeek(value[, tzinfo=None])
    TruncDay(value[, tzinfo=None])
    TruncHour(value[, tzinfo=None])
    TruncMinute(value[, tzinfo=None])
    TruncSecond(value[, tzinfo=None])
```